In [6]:
import pandas as pd
import numpy as np
import mysql.connector as mysql

db = mysql.connect(
    host = "localhost",
    user = "editekk",
    passwd = "editekk",
    database = "RidgeMeasurements"
)
cursor = db.cursor()

In [37]:
cursor.execute("DROP TABLE pedicle_screw_averages")

In [38]:
exec = str(
    "CREATE TABLE pedicle_screw_averages " 
    # "(id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, "
    "(sample_id VARCHAR(15), "
    "fluence FLOAT(24), " 
    "dims VARCHAR(15), " 
    "batch VARCHAR(15), " 
    "radial_loc VARCHAR(3), "
    "linear_loc VARCHAR(7), "
    "thread_loc VARCHAR(7), "
    "wavlen_avg_validate FLOAT(10), "
    "wavelen_avg_new FLOAT(10))")
cursor.execute(exec)

In [39]:
cursor.execute("SELECT sample_id, radial_loc, linear_loc, thread_loc FROM pedicle_screw_summary")
loc_data = cursor.fetchall()

cursor.execute("SELECT wavelength, line_len, line_ints FROM pedicle_screw_summary")
wavlen_data = cursor.fetchall()

In [40]:
def get_vals_str(i):
    # TODO: CHECKME:
    return str(
        "SELECT line_count, "
        "sample_id, "
        "fluence, "
        "dims, "
        "batch, "
        "radial_loc, "
        "linear_loc, "
        "thread_loc "
        "FROM pedicle_screw_summary "
        "WHERE id=" + str(i+1))

def insert_vals(fet, avg_valid, avg_new):
    _fetch = fet[0][1:] + tuple([avg_valid, avg_new])
    exec = str(
            "INSERT INTO pedicle_screw_averages "
            " VALUES " + str(_fetch)
    )
    cursor.execute(exec)
    db.commit()


In [41]:
sum_wl = wavlen_data[0][0]
sum_ll = wavlen_data[0][1]
sum_li = wavlen_data[0][2]
line_count = 1
len_loc_data = len(loc_data)
for i in range(len_loc_data-1):
    if loc_data[i] == loc_data[i+1]:
        sum_wl += wavlen_data[i+1][0]
        sum_ll += wavlen_data[i+1][1]
        sum_li += wavlen_data[i+0][2]
        line_count += 1
    else:
        cursor.execute(get_vals_str(i))
        fetch = cursor.fetchall()
        avg_vd = sum_wl / line_count
        avg_new = sum_ll / sum_li
        insert_vals(fetch, avg_vd, avg_new)
        sum_wl = wavlen_data[i+1][0]
        sum_ll = wavlen_data[i+0][1]
        sum_li = wavlen_data[i+0][2]
        line_count = 1

cursor.execute("SELECT line_count FROM pedicle_screw_summary ORDER BY ID DESC LIMIT 1")
last_count = cursor.fetchall()
cursor.execute(get_vals_str(len_loc_data-1))
last_row = cursor.fetchall()
avg_vd = sum_wl / last_count[0][0]
avg_new = sum_ll / sum_li
insert_vals(last_row, avg_vd, avg_new)

In [51]:
cursor.execute("SELECT DISTINCT sample_id FROM pedicle_screw_averages")
for s in cursor.fetchall():
    cursor.execute(str("SELECT COUNT(sample_id) FROM pedicle_screw_averages WHERE sample_id='" + s[0] + "' AND radial_loc='r1'"))
    if cursor.fetchall()[0][0] != 6:
        cursor.execute(str("UPDATE pedicle_screw_averages SET quality_ctrl='FAIL' WHERE sample_id='" + s[0] + "' AND radial_loc='r1'"))
        db.commit()